**46. Si decimos que la ubicación de una categoría es el promedio de la latitud y longitud
de sus contenidos geolocalizados que son miembros de ella (si es que tiene):
¿Cuales son las dos categorías más cercanas? (⭐⭐⭐)**

In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Para este ejercicio voy a utilizar el csv **categorylinks** y **geo_tags**

In [ ]:
id='1onmfzlzZxs3YFo5sXKZmft20c2kSqrJ3'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('categorylinks.csv')

id2='1qLu79VkVJxWp9i4Pnq9cxSLjW6Nipxp9'
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('geo_tags.csv')

Hay varias columnas de los csv que no voy a necesitar y así que directamente no las leo (ahorro de memoria)

In [ ]:
category = pd.read_csv('categorylinks.csv', encoding='latin-1', usecols=["cl_from", "cl_to"],\
                            dtype={'cl_to':'category'})
category.head()

,cl_from,cl_to
0,5,Wikipedia:Mantenimiento
1,5,Wikipedia:PÃÂ¡ginas_con_enlaces_mÃÂ¡gicos_de...
2,7,Andorra
3,7,Instituciones_patrocinadoras_de_equipos_ciclistas
4,7,Wikipedia:ArtÃÂ­culos_con_datos_locales


In [ ]:
geo_tags = pd.read_csv('geo_tags.csv', encoding='latin-1', usecols=["gt_page_id", "gt_globe", "gt_lat", "gt_lon"],\
                            dtype={'gt_globe':'category'})
geo_tags.head()

,gt_page_id,gt_globe,gt_lat,gt_lon
0,4328020,earth,41.5075,-5.8175
1,4670424,earth,13.7380,-89.2920
2,4670424,earth,13.7020,-89.2080
3,4670424,earth,13.4000,-89.7500
4,4670424,earth,12.8500,-88.0000


Como solo me va a interesar el contenido geolocalizado, hago un merge por ids entre estos csv:

In [ ]:
category_geo = category.merge(geo_tags, how='inner', left_on='cl_from', right_on='gt_page_id')
category_geo.head()

,cl_from,cl_to,gt_page_id,gt_globe,gt_lat,gt_lon
0,7,Andorra,7,earth,42.5,1.5
1,7,Instituciones_patrocinadoras_de_equipos_ciclistas,7,earth,42.5,1.5
2,7,Wikipedia:ArtÃÂ­culos_con_datos_locales,7,earth,42.5,1.5
3,7,Wikipedia:ArtÃÂ­culos_con_datos_por_trasladar...,7,earth,42.5,1.5
4,7,Wikipedia:ArtÃÂ­culos_con_enlaces_externos_rotos,7,earth,42.5,1.5


In [ ]:
len(category_geo)

3042010

Podemos ver que del contenido que teniamos en categorylinks.csv hay **3042010** geolocalizados

Calculo la ubicación de cada categoría, haciendo el promedio entre las latitudes y longitudes de sus correspondientes contenidos geolocalizados.

In [ ]:
grouped = category_geo.groupby(['cl_to']).agg({'gt_lat':['mean'], 'gt_lon':['mean'] })
grouped.columns = grouped.columns.get_level_values(0) + '_' + grouped.columns.get_level_values(1)
grouped = grouped.dropna()
grouped.reset_index(inplace=True)
grouped = grouped.rename(columns = {'cl_to': 'Categoria', 'gt_lat_mean':'lat','gt_lon_mean':'lon'})
grouped

,Categoria,lat,lon
0,10,41.900600,12.478700
1,100_montaÃÂ±as_famosas_de_JapÃÂ³n,35.730490,138.206041
2,101,44.794466,25.337350
3,1017,51.719200,8.755560
4,101_a._C.,45.316667,8.416667
...,...,...,...
108037,Sitios_Ramsar_de_Noruega,60.783333,11.116667
108038,Sitios_arqueolÃÂ³gicos_del_distrito_de_San_Luis,-12.079180,-76.999693
108039,Tost,42.272830,1.384922
108040,Valles_de_Piamonte,44.410000,7.300000


In [ ]:
grouped.duplicated(['lat', 'lon']).sum()

9242

Se da el caso de que hay categorías que quedaron en la misma posición, por lo tanto van a quedar varios con distancia cero. Para evitar esto voy a eliminar los "repetidos" y quedarme solo con el primero

In [ ]:
grouped = grouped.drop_duplicates(['lat', 'lon'], keep='first').reset_index()
grouped = grouped.drop(['index'], axis=1)
grouped

,Categoria,lat,lon
0,10,41.900600,12.478700
1,100_montaÃÂ±as_famosas_de_JapÃÂ³n,35.730490,138.206041
2,101,44.794466,25.337350
3,1017,51.719200,8.755560
4,101_a._C.,45.316667,8.416667
...,...,...,...
98795,Resistencia_al_Imperio_britÃÂ¡nico,22.266700,114.150000
98796,SK_VorwÃÂ¤rts_Steyr,48.037667,14.410361
98797,Sitios_Ramsar_de_Noruega,60.783333,11.116667
98798,Sitios_arqueolÃÂ³gicos_del_distrito_de_San_Luis,-12.079180,-76.999693


Hice un reset index para que la columna de la misma quede sin "saltos" y así evitar posibles advertencias/errores más adelante

Voy a utilizar un BallTree para poder buscar el vecino más cercano de cada usuario. (Autorizado por Nati)

In [ ]:
from sklearn.neighbors import BallTree

query_lats = grouped['lat']
query_lons = grouped['lon']

bt = BallTree(np.deg2rad(grouped[['lat', 'lon']].values), metric='haversine')
distances, indices = bt.query(np.deg2rad(np.c_[query_lats, query_lons]), k=2)

El array distances tendrá las distancias de los 2 vecinos más cercanos, donde siempre el primer valor será 0 (distancia a el mismo) y el segundo tendrá un valor.

El array indices tendrá también 2 valores, el primero será el mismo y el 2do será el indice del vecino más cercano (distinto de él)

Paso las distancias a KM multiplicando por 6371 que es el radio de la tierra en km.

In [ ]:
radio = 6371
for i in range(0, len(distances)):
  distances[i][1] = distances[i][1] * radio

print(distances)

[[ 0.          0.04883094]
 [ 0.          8.51198895]
 [ 0.         17.65677683]
 ...
 [ 0.         12.65607797]
 [ 0.          0.21768522]
 [ 0.          2.77179275]]


In [ ]:
print(indices)

[[    0 22082]
 [    1 61114]
 [    2 63702]
 ...
 [98797 70057]
 [98798 61110]
 [98799 53498]]


Vamos a dividir estos datos en columnas individuales y las cargamos al dataframe.

Los datos de las primeras columnas los voy a descartar porque contienen datos de cada categoría a si misma y no me sirven.

In [ ]:
d_1, d_2 = np.split(distances, 2, axis=1)
i_1, i_2 = np.split(indices, 2, axis=1)
grouped['vecino_mas_cercano'] = i_2
grouped['distancia'] = d_2
grouped.head()

,Categoria,lat,lon,vecino_mas_cercano,distancia
0,10,41.900600,12.478700,22082,0.048831
1,100_montaÃÂ±as_famosas_de_JapÃÂ³n,35.730490,138.206041,61114,8.511989
2,101,44.794466,25.337350,63702,17.656777
3,1017,51.719200,8.755560,95087,0.039664
4,101_a._C.,45.316667,8.416667,19552,3.323988


Filtro por la distancia minima:

In [ ]:
grouped[(grouped['distancia'] == grouped['distancia'].min())]

,Categoria,lat,lon,vecino_mas_cercano,distancia
12311,Tratados_de_RepÃÂºblica_Dominicana,15.114815,-71.541667,12311,0.0
61785,EspaÃÂ±a_en_1246,37.769722,-3.788889,71527,0.0
71527,Historia_de_JaÃÂ©n,37.769722,-3.788889,71527,0.0
96930,Frontera_Colombia-RepÃÂºblica_Dominicana,15.114815,-71.541667,12311,0.0


Por alguna razón que desconozco quedaron algunos datos duplicados, vamos a descartarlos:

In [ ]:
grouped = grouped[(grouped['distancia'] > 0)]

Filtro nuevamente por la distancia minima:

In [ ]:
grouped[(grouped['distancia'] == grouped['distancia'].min())]

,Categoria,lat,lon,vecino_mas_cercano,distancia
96829,Frontera_Colombia-HaitÃÂ­,14.884722,-73.986111,96835,3.536615e-13
96835,HaitÃÂ­_en_1978,14.884722,-73.986111,96829,3.536615e-13


Podemos ver que las categorías mas cercanas con una distancia muuuuy chica son: 
**Frontera_Colombia-HaitÃÂ­	 Y HaitÃÂ­_en_1978**

Y si hacemos un filtrado de distancias un poco mayor a 0:

In [ ]:
grouped = grouped[(grouped['distancia'] > 0.01)]
grouped[(grouped['distancia'] == grouped['distancia'].min())]

,Categoria,lat,lon,vecino_mas_cercano,distancia
39267,Edificios_bancarios_de_Argentina_del_siglo_XXI,-34.605694,-58.372639,39721,0.010001
39721,Grupo_Financiero_Galicia,-34.605764,-58.372569,39267,0.010001


Estos usuarios por ejemplo estarían a una distancia de 0,01 km que equivale a 10 metros